<h1>Step 1: Collecting Data</h1>

---

In [9]:
! pip install requests-html

In [10]:
! pip install lxml[html_clean]

In [11]:
! pip install selenium


In [12]:
from requests_html import HTML
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import urllib.robotparser
import pandas as pd
import datetime
import time
import logging
from selenium.common.exceptions import WebDriverException
from selenium import webdriver

Check file robots.txt

In [13]:
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://www.worldweatheronline.com//robots.txt')
rp.read()
rp.can_fetch('*', 'https://www.worldweatheronline.com/ho-chi-minh-city-weather-history/vn.aspx')

True

Dùng thư viện Selenium để tương tác với trang web và requests_html để parse html 

In [14]:
! pip install webdriver-manager

In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# Set up the ChromeDriver using webdriver_manager to handle driver installation
service = Service(ChromeDriverManager().install())

# Initialize the browser
browser = webdriver.Chrome(service=service)

# Open the webpage
browser.get('https://www.worldweatheronline.com/ho-chi-minh-city-weather-history/vn.aspx')


2025-05-09 08:52:27,141 - ====== WebDriver manager ======
2025-05-09 08:52:28,675 - Get LATEST chromedriver version for google-chrome
2025-05-09 08:52:29,026 - Get LATEST chromedriver version for google-chrome
2025-05-09 08:52:29,321 - Driver [C:\Users\doanh\.wdm\drivers\chromedriver\win64\136.0.7103.92\chromedriver-win32/chromedriver.exe] found in cache


In [16]:
import logging

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

# Open CSV file
with open('./weatherHCM.csv', 'w', encoding='utf-8') as f:
    f.write('Date\tTime\tWeather\tTemp\tFeels\tWind\tGust\tRain\tHumidity\tCloud\tPressure\tVis\n')

    date = datetime.datetime(2009, 1, 1)
    end_date = datetime.datetime(2020, 12, 21)
    date_col = ''
    record = ['Time', 'Weather', 'Temp', 'Feels', 'Wind', 'Gust', 'Rain', 'Humidity', 'Cloud', 'Pressure', 'Vis']
    n_cols = len(record)

    while date < end_date:
        date_str = date.strftime('%Y-%m-%d')
        logging.info(f"Processing date: {date_str}")

        try:
            # Wait for the date input field and interact with it
            WebDriverWait(browser, 10).until(
                EC.visibility_of_element_located((By.ID, 'ctl00_MainContentHolder_txtPastDate'))
            )

            # Adjust date
            input_date = browser.find_element(By.ID, 'ctl00_MainContentHolder_txtPastDate')
            browser.execute_script("arguments[0].value = arguments[1];", input_date, date_str)
            
            submit_date = browser.find_element(By.ID, 'ctl00_MainContentHolder_butShowPastWeather')
            submit_date.click()
            
            # Wait for page loading (use WebDriverWait instead of time.sleep)
            WebDriverWait(browser, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'col'))
            )

            # Parse the page source
            html = HTML(html=browser.page_source)
            yesterday = date_col

            # Try to find the card header
            card_header = html.find('.card-header', first=True)
            if card_header is None:
                logging.error(f"Could not find '.card-header' for date {date_str}, skipping...")
                date += datetime.timedelta(days=1)
                continue

            date_col = card_header.find('h5', first=True).text + ' ' + str(date.year)

            if date_col == yesterday:
                logging.info(f"Duplicate data for date {date_str}, skipping...")
                continue

            date += datetime.timedelta(days=1)
            cols = html.find('.col')[20:108]

            i = 0
            for col in cols:
                record[i % n_cols] = col.text
                i += 1

                if i % n_cols == 0:
                    f.write(f'{date_col}\t{record[0]}\t{record[1]}\t{record[2]}\t{record[3]}\t{record[4]}\t{record[5]}\t{record[6]}\t{record[7]}\t{record[8]}\t{record[9]}\t{record[10]}\n')

        except Exception as e:
            logging.error(f"Error processing date {date_str}: {e}")
            date += datetime.timedelta(days=1)
            continue

browser.quit()

2025-05-09 08:52:35,005 - Processing date: 2009-01-01
2025-05-09 08:52:35,011 - Error processing date 2009-01-01: name 'WebDriverWait' is not defined
2025-05-09 08:52:35,012 - Processing date: 2009-01-02
2025-05-09 08:52:35,013 - Error processing date 2009-01-02: name 'WebDriverWait' is not defined
2025-05-09 08:52:35,014 - Processing date: 2009-01-03
2025-05-09 08:52:35,015 - Error processing date 2009-01-03: name 'WebDriverWait' is not defined
2025-05-09 08:52:35,017 - Processing date: 2009-01-04
2025-05-09 08:52:35,018 - Error processing date 2009-01-04: name 'WebDriverWait' is not defined
2025-05-09 08:52:35,019 - Processing date: 2009-01-05
2025-05-09 08:52:35,020 - Error processing date 2009-01-05: name 'WebDriverWait' is not defined
2025-05-09 08:52:35,022 - Processing date: 2009-01-06
2025-05-09 08:52:35,022 - Error processing date 2009-01-06: name 'WebDriverWait' is not defined
2025-05-09 08:52:35,024 - Processing date: 2009-01-07
2025-05-09 08:52:35,026 - Error processing dat